### RasCAL-1 to RAT

RasCAL-1 (R1) project structs can be converted to RAT Project classes, and vice versa. This is done via the functions `r1_to_project_class` and `project_class_to_r1`.

Converting from R1 to a `Project` is very simple. We use the example R1 project in the file `R1monolayerVolumeModel.mat`, which is a project for analysing a monolayer of DSPC with various deuterations (tail-deuterated, head-deuterated, fully deuterated, hydrogenated)

Simply give the file path to the function `r1_to_project_class`, and it returns a RAT `Project` that you can use exactly like any other.

In [1]:
from RATapi.utils.convert import r1_to_project_class

project = r1_to_project_class("R1monolayerVolumeModel.mat")
print(project)

Name: ----------------------------------------------------------------------------------------------

monolayerVolumeModel

Calculation: ---------------------------------------------------------------------------------------

normal

Model: ---------------------------------------------------------------------------------------------

custom layers

Geometry: ------------------------------------------------------------------------------------------

air/substrate

Parameters: ----------------------------------------------------------------------------------------

+-------+---------------------+------+--------------------+-------+------+
| index |         name        | min  |       value        |  max  | fit  |
+-------+---------------------+------+--------------------+-------+------+
|   0   | Substrate Roughness | 1.0  | 2.9979642781948908 |  8.0  | True |
|   1   |  Area per molecule  | 47.0 | 53.052680457664785 | 100.0 | True |
|   2   |    Head Thickness   | 7.0  | 12.2763338367799

Note that there are various features of RAT which do not feature in RasCAL-1, such as `prior_type`, `mu` and `sigma` for parameters. These are given sensible default values (again e.g. for parameters, `prior_type = uniform`, `mu = 0.0`, `sigma=inf`), but you may change these if you would like to use these new features:

In [2]:
project.parameters["Head Thickness"].prior_type = 'gaussian'
project.parameters["Theta"].mu = 2.0
project.parameters["Area per molecule"].sigma = 50.0
# etc...
print(project.parameters)

+-------+---------------------+------+--------------------+-------+------+
| index |         name        | min  |       value        |  max  | fit  |
+-------+---------------------+------+--------------------+-------+------+
|   0   | Substrate Roughness | 1.0  | 2.9979642781948908 |  8.0  | True |
|   1   |  Area per molecule  | 47.0 | 53.052680457664785 | 100.0 | True |
|   2   |    Head Thickness   | 7.0  | 12.276333836779942 |  20.0 | True |
|   3   |        Theta        | 0.0  | 28.870541049836262 |  50.0 | True |
+-------+---------------------+------+--------------------+-------+------+


Also note that any custom files must be available to RAT. By default, RAT will assume these files are in the same directory that you are running RAT from, but if they are elsewhere you may change the relevant file location: 

In [3]:
# e.g. if our model is in the directory `my_models/`
project.custom_files[0].filename = "my_models/Model_IIb.m"

As well as MATLAB functions, RAT can also run custom files provided in Python or C++ format. This may be beneficial if you do not have access to MATLAB, do not have access to the custom files from your old RasCAL project, or find it more convenient to use Python. This is done similarly to changing the file path: if we have a function defined in the Python custom file `Model_IIb.py`:

In [4]:
project.custom_files[0].filename = "Model_IIb.py"
project.custom_files[0].language = 'python'

### RAT to RasCAL-1

To demonstrate the other way around, we will use the DSPC lipid bilayer model project from another tutorial.

In [5]:
from RATapi.examples import DSPC_standard_layers
lipid_bilayer_project = DSPC_standard_layers()[0]
print(lipid_bilayer_project)

Starting RAT ───────────────────────────────────────────────────────────────────────────────────────────────────────────

Elapsed time is 0.002 seconds

Finished RAT ───────────────────────────────────────────────────────────────────────────────────────────────────────────

Name: ----------------------------------------------------------------------------------------------

original_dspc_bilayer

Calculation: ---------------------------------------------------------------------------------------

normal

Model: ---------------------------------------------------------------------------------------------

standard layers

Geometry: ------------------------------------------------------------------------------------------

substrate/liquid

Parameters: ----------------------------------------------------------------------------------------

+-------+-------------------------+----------+-----------+----------+-------+
| index |           name          |   min    |   value   |   max    |  

`project_class_to_r1` takes parameters `project` and `filename`, which are the `Project` object and filename for the produced .mat file respectively. This .mat file can then be loaded into RasCAL-1.

Alternatively, if one sets `return_struct=True`, the struct is returned as a Python dictionary instead of being saved.

Note that a MATLAB engine is used to save the project to a .mat file, so the Python library `matlabengine` must be installed.

In [6]:
from RATapi.utils.convert import project_class_to_r1
from pprint import pp  # for printing the struct

# save to a file called lipid_bilayer.mat
project_class_to_r1(lipid_bilayer_project, filename="lipid_bilayer.mat")

# return as a Python dictionary
struct = project_class_to_r1(lipid_bilayer_project, return_struct=True)
pp(struct)

{'name': 'original_dspc_bilayer',
 'module': {'type': 'standard layers',
            'experiment_type': 'Solid / Liquid',
            'name': ''},
 'paramnames': ['Substrate Roughness',
                'Oxide Thickness',
                'Oxide SLD',
                'SAM Tails Thickness',
                'SAM Tails SLD',
                'SAM Tails Hydration',
                'SAM Roughness',
                'CW Thickness',
                'CW SLD',
                'SAM Heads Thickness',
                'SAM Heads SLD',
                'SAM Heads Hydration',
                'Bilayer Heads Thickness',
                'Bilayer Heads SLD',
                'Bilayer Roughness',
                'Bilayer Tails Thickness',
                'Bilayer Tails SLD',
                'Bilayer Tails Hydration',
                'Bilayer Heads Hydration',
                'CW Hydration',
                'Oxide Hydration'],
 'params': [3.0,
            19.54,
            3.39e-06,
            22.66,
         